In [26]:
import ast

In [2]:
def path_of_file(lib_path=None):
    if lib_path is not None:
        import sys
        import os
        sys.path.insert(0,lib_path)
        with open("input.py", "r") as f:
            code = f.read() #
        tree = ast.parse(code)
    return tree

In [6]:
class WildcardImportChecker(ast.NodeVisitor):
    def visit_ImportFrom(self, node):
        for alias in node.names:
            self._file_module[f"{alias.name}"]=f'{node.module}'
        self.generic_visit(node)
    def set_file_module(self, module):
        self._file_module = module
        
def import_module(file_path):
    tree = path_of_file(file_path)
    file_module={}
    visitor = WildcardImportChecker()
    visitor.set_file_module(file_module)
    visitor.visit(tree)
    return file_module
    
h=r"C:/Users/Dell/Desktop/storage/timeline/2023/july/auto access relate code/input.py"    
print(import_module(h))

{'ModuleReloader': 'abcL', 'main': 'abcd', 'bidderAndBidReadOnly': 'abcd'}


In [11]:
class PrintToCallFunctionTransformer(ast.NodeTransformer):
    def visit_Call(self, node):
        if node.func.id!='print':
            self._function+=[f'{node.func.id}']
        self.generic_visit(node)
    def set_function(self,id):
        self._function = id

def import_function_name(file_path):
    tree = path_of_file(file_path)
    function=[]
    transformer = PrintToCallFunctionTransformer()
    transformer.set_function(function)
    transformer.visit(tree)
    return function

h=r"C:/Users/Dell/Desktop/storage/timeline/2023/july/auto access relate code/input.py"    
print(import_function_name(h))

['bidderAndBidReadOnly', 'main']


In [25]:
def find_assign_nodes(source_code):
    tree = ast.parse(source_code)
    assign_nodes = [node for node in ast.walk(tree) if isinstance(node, ast.Assign)]
    return assign_nodes

In [ ]:
def exclude_imports_functions(code_str):
    parsed_ast = ast.parse(code_str)
    def reconstruct_code(node):
        if isinstance(node, (ast.Import, ast.ImportFrom)):
            return ""
        return ast.unparse(node)
    filtered_code = "\n".join(reconstruct_code(node) for node in parsed_ast.body)
    return filtered_code

In [33]:
def call_function(function):
    x_list=set()
    assign_nodes = find_assign_nodes(function)
    for assign_node in assign_nodes:
        targets = [target.id for target in assign_node.targets if isinstance(target, ast.Name)]
        value = ast.unparse(assign_node.value).strip()
        x=''
        if len(targets)==0:
            for j in value:
                if j=='(':
                    x_list.add(x)
                    break
                else:
                    x+=j
    return x_list

call_function(function)

{'bidderAndBid', 'bidderAndBidReadOnly'}


In [29]:
import ast
import inspect

def fileOpen():
    try:
        file=open('output.py','a')
        return file 
    except:
        pass

def newFile(func_name):
    x_list=set()
    for i in func_name:
        module = __import__(str(file_module[i]))#file name 
        source_code = inspect.getsource(getattr(module,i))#def func name
        file=fileOpen()
        file.write(source_code)
        file.close()



    if len(x_list)!=0:
        source = (open("output.py", "r").read())
        functions = [f.name for f in ast.parse(source).body
             if isinstance(f, ast.FunctionDef)]

    remain_def=x_list.difference(set(functions))
    for j in remain_def:
        source_code = inspect.getsource(getattr(module,j))#def func name
        file=fileOpen()
        file.write(source_code)
        file.close()

newFile(func_name) 

filtered_code = exclude_imports_functions(code)
file=fileOpen()
file.write(filtered_code)
file.close()

